# CS 201: Recursion

In [1]:
(require racket)
(require racket/base)

## Recursion, Glorious, Recursion

See [recursion.rkt](recursion.rkt) and try out the trace facility

modulo vs remainder See [modulo.rkt](modulo.rkt) (Also, quotient vs /)

Recursive procedures that take out lists as arguments and return lists as values

See [racket4.rkt](racket4.rkt)

Recall from above that we have list predicates: `empty?`, `null?`, `list?`, list selectors: `first`, `rest`, `car`, `cdr`, and a list constructure: `cons`.

There is a built-in procedure `(length lst)` that takes a list `lst` as its argument and returns the number of top-level elements in lst. For example we have the following

In [2]:
(length '())

0

In [3]:
(length '(a b c))

3

In [4]:
(length '((1 2) (3 4 5)))

2

We'll write our own recursive version of length, `(our-length lst)`. As we saw , we need one or more base cases. A natural one for an input list is the empty list, '(). If the input list is empty, we can immediately return the value 0 for `our-length`. We also need one or more recursive cases that "move" the input towards a base case. In the case of an input list, removing one element from the list (by using `rest` or `cdr`) is a natural candidate, since it moves towards the empty list as a base case. As we think about how to construct the procedure, it may be helpful to consider an example.

In [5]:
(define lst '(17 24 6))

In [6]:
(rest lst)

'(24 6)

In [7]:
(length (rest lst))

2

In [8]:
(first lst)

17

In [9]:
(length lst)

3

In this case, if the recursive call `(our-length (rest lst))` returns a correct value, we just need to add 1 to it to get a correct answer for `(our-length lst)`. We don't actually seem to need the value of `(first lst)` at all. We are led to define the following procedure.

In [10]:
(define (our-length lst)
  (if (empty? lst)
      0
      (+ 1 (our-length (rest lst)))))

We can draw a tree of the recursive calls of `(our-length '(17 24 6))` to help us understand how this works.

    (our-length '(17 24 6))
     /  |     \
    +   1   (our-length '(24 6))
             /  |     \
            +   1    (our-length '(6))
                      /  |     \
                     +   1    (our-length '())

Notice that each successive recursive call has an argument that is one element shorter than its predecessor, until we reach the base case of the empty list '(). Then there is no further recursive call, and the procedure just returns 0, leading to the following sequence of return values (from the bottom up.)

    (our-length '(17 24 6)) => 3
     /  |     \
    +   1   (our-length '(24 6)) => 2
             /  |     \
            +   1    (our-length '(6)) => 1
                      /  |     \
                     +   1    (our-length '()) => 0

Note that the if expression tests `(empty? lst)` -- it would be equivalent to test `(null? lst)` or `(equal? lst '())`.

This procedure is not tail-recursive, because the value of the recursive call, `(our-length (rest lst))` is modified, that is, has 1 added to it, before being returned as the value of `(our-length lst)`. We discuss tail-recursion below.


In [11]:
(our-length '(17 24 6))

3

In [12]:
(require racket/trace)

In [13]:
(trace our-length)

In [14]:
(our-length '(17 24 6))

>(our-length '(17 24 6))
> (our-length '(24 6))
> >(our-length '(6))
> > (our-length '())
< < 0
< <1
< 2
<3


3

## Testing whether an element is in a list

There is a built-in procedure `(member item lst)` to test whether an arbitrary item is `equal?` to a top-level element of the list `lst`. If item is not `equal?` to any of the top-level elements of `lst`, then this procedure returns `#f`. If item is equal to some top-level element of `lst`, then this procedure returns a list with all the remaining elements of `lst` from the first one `equal?` to `item` until the end. As examples, we have the following.

In [15]:
(member 2 '(1 2 3))

'(2 3)

In [16]:
(member 2 '(1 2 3 2 1))

'(2 3 2 1)

In [17]:
(member 4 '(1 2 3))

#f

Note that this is not a predicate, because it does not always return `#t` or `#f`. We'll write (as a recursive procedure) a predicate version of member, namely `(member? item lst)`. If item is `equal?` to a top-level element of `lst`, then `(member? item lst)` evaluates to `#t`; otherwise, it evaluates to `#f`. We can define a trivial version using member.

In [18]:
(define (member? item lst)
  (if (member item lst) #t #f))

In [19]:
(member? 2 '(1 2 3))

#t

In [20]:
(member? 2 '(1 2 3 2 1))

#t

In [21]:
(member? 4 '(1 2 3))

#f



Because the input `lst` is a list, a reasonable base case is when `lst` is an empty list. In this case, lst contains no top-level elements, so item cannot be `equal?` to one of them, and the procedure should return `#f`. Thinking about the first example above, we have `item => 2` and `lst => '(1 2 3)`.

    (first lst) => 1
    (rest lst) => '(2 3)
    (member? item (rest lst)) => #t   (if it returns the correct answer)

So there should be a recursive case in which we evaluate `(member? item (rest lst))` and return the value that it returns. However, this isn't the only case; we have the case in which item is actually `equal?` to `(first lst)`. This is another base case, which should return the value `#t`. We are led to the following procedure definition.


In [22]:
(define (member? item lst)
  (cond
    [(empty? lst) #f]
    [(equal? item (first lst)) #t]
    [else (member? item (rest lst))]))

In [23]:
(define (ifmember? item lst)
  (if (empty? lst) #f
    (if (equal? item (first lst)) #t
      (ifmember? item (rest lst)))))

In [24]:
(ifmember? 2 '(1 2 3))

#t



We used a `cond` special form to avoid a situation of nested if expressions. There are two base cases: `lst` is empty, and the answer is `#f`, and the list is non-empty and item is `equal?` to the first element of `lst`, when the answer is `#t`. If neither of these cases applies, then we need to continue to search for item in the rest of the list. The answer (`#t` or `#f`) that the recursive call returns will be the answer to whether `(member? item lst)` should be `#t` or `#f`. We can draw the tree of recursive calls for `(member? 24 '(17 24 6))` as follows.

    (member? 24 '(17 24 6))
        |
    (member? 24 '(24 6))

There are no further recursive calls because 24 is `equal?` to `(first lst)` in this case, and `#t` is returned. The return value just propagates upward as follows.

    (member? 24 '(17 24 6)) => #t
        |
    (member? 24 '(24 6)) => #t
    

Note that our member? procedure is tail-recursive: the value of the recursive call is returned (unmodified) as the value of the whole procedure.

In [25]:
(not 1)

#f

In [26]:
(not '())

#f

## A procedure that takes a list as input and returns a list

Now we write a procedure `(double-each lst)` that takes a list lst of numbers and returns the list consisting of twice each number in the original list. As an example we have the following.

    (double-each '(17 24 6)) => '(34 48 12)

Once again, the empty list is a reasonable base case. We have to decide what we want `(double-each '())` to be. We choose the result to be `'()`, which consists of twice each number in `'()`. Thinking about the example, if `lst => '(17 24 6)`, we have the following.

    (first lst) => 17
    (rest lst) => '(24 6)
    (double-each (rest lst)) => '(48 12)  (assuming it works correctly)


For the recursive case, we can call `(double-each (rest lst))`, and get a list containing twice each of the numbers in the rest of the list. To make this into the desired result, all we have to do is to include twice (`first lst)` at the front of the list. The list constructor `(cons item lst)` returns the list `equal?` to `lst` with item included as the first element. We may now write the complete procedure as follows.

In [27]:
(define (double-each lst)
  (if (null? lst) '()
      (cons (* 2 (first lst))
             (double-each (rest lst)))))

In [28]:
(double-each '(1 2 3))

'(2 4 6)

In [29]:
(trace double-each)

In [30]:
(double-each '(17 24 6))

>(double-each '(17 24 6))
> (double-each '(24 6))
> >(double-each '(6))
> > (double-each '())
< < '()
< <'(12)
< '(48 12)
<'(34 48 12)


'(34 48 12)

Note that we could get by with an if expression because there were only two cases: the base case and the recursive case. We can draw the tree of recursive calls for the application `(double-each '(17 24 6))` as follows.

    (double-each '(17 24 6))
     /   |    \
    cons  34  (double-each '(24 6))
              /   |    \
            cons  48  (double-each '(6))
                       /   |    \
                     cons  12  (double-each '())

There are no further recursive calls because we reached the base case `(lst => '())`, which returns `'()`. Then the returns propagate up the tree as follows.

    (double-each '(17 24 6)) => '(34 48 12)
     /   |    \
    cons  34  (double-each '(24 6)) => '(48 12)
              /   |    \
            cons  48  (double-each '(6)) => '(12)
                       /   |    \
                     cons  12  (double-each '()) => '()

Note that this procedure is not tail-recursive, because the value returned by the recursive call is modified (by having twice `(first lst)` `cons`'ed on the front before being returned as the value of the whole procedure.

## Deep recursion

See add-tree in [racket5.rkt](racket5.rkt)

So far we have seen recursion on numbers, from n to n-1, or from `(quotient n 10)`, and "flat" recursion on lists, which proces `(first lst)` in some way and does a recursive call on `(rest lst)`. Now we'll consider a procedure that does "deep recursion", in which the procedure is called recursively on sublists, and sublists of sublists, and so on. There is a built-in procedure `(flatten value)`. Here are some examples of it.

In [31]:
(require racket)
(require racket/base)

In [32]:
(flatten 17)

'(17)

In [33]:
(flatten '(17 24 6))

'(17 24 6)

In [34]:
(flatten '((1 2) (4 (1 2)) 3))

'(1 2 4 1 2 3)

In [35]:
(flatten '(() ()))

'()

If the argument value is not a list, the return value is a list containing it as the only element. If the argument value is the empty list, the return value is the empty list. However, if the argument value is a non-empty list, the result is obtained by appending the result of flattening `(first value)` to the result of flattening `(rest lst)`. We'll write our own version of flatten, `(o-f value)`.

In [36]:
(define (o-f value)
  (cond
    [(not (list? value)) (list value)]
    [(empty? value) '()]
    [else
      (append (o-f (first value))
              (o-f (rest value)))]))

Note that here we've used the built-in list constructor list. This takes an arbitrary finite number of arguments, evaluates all of them, and makes a list of the values. Thus, `(list value)` is equivalent to `(cons value '())`. Note the contrast with quote ('), in the following.

In [37]:
(car '('a 1))

''a

In [38]:
(car (quote ((quote a) 1)))

''a

In [39]:
(list 'a 1)

'(a 1)

In [40]:
(list (+ 1 2) (* 2 3))

'(3 6)

In [41]:
'((+ 1 2) (* 3 6))

'((+ 1 2) (* 3 6))

In [42]:
(quote ((+1 2) (* 2 3)))

'((1 2) (* 2 3))

In the case of list, the expressions `(+ 1 2)` and `(* 2 3)` are evaluated, and the list of their values returned. In the case of quote ('), evaluation of the sub-expressions of the expression is inhibited, and the value of `'((+ 1 2) (* 3 6))` is a list containing two lists, the first with the identifier '+, the number 1 and the number 2, and the second with the identifier '*, the number 3 and the number 6.

We may draw the tree of recursive calls for an application like `(o-f '((1 2) 3)).`

In [43]:
(require racket/trace)

In [44]:
(trace o-f)

In [45]:
(o-f `((1 2) 3))

>(o-f '((1 2) 3))
> (o-f '(1 2))
> >(o-f 1)
< <'(1)
> >(o-f '(2))
> > (o-f 2)
< < '(2)
> > (o-f '())
< < '()
< <'(2)
< '(1 2)
> (o-f '(3))
> >(o-f 3)
< <'(3)
> >(o-f '())
< <'()
< '(3)
<'(1 2 3)


'(1 2 3)



              (o-f '((1 2) 3))
           _______________________
           /    |                 \
          /     |                  \
         /      |                   \
        /       |                    \ 
    append  (o-f '(1 2))              (o-f '(3))
           /    |     \               /    |    \
      append  (o-f 1) (o-f '(2))  append (o-f 3) (o-f '())
                     /    |    \
                append (o-f 2) (o-f '()) 

All the leaves are base cases (either the empty list or a non-list value), and they are appended and returned up the tree of recursive calls to yield `(o-f '((1 2) 3)) => '(1 2 3)`.

Going forward, deep recursion will appear in many guises, to traverse or process more complex recursive data structures. It is your friend. Like it on Facebook. (BTW, Facebook is basically a huge graph which is a mammoth recursive data structure.)


## add-tree deep recursion

We first define a variable with a tree structure.

In [46]:
(define s '((2 3 (4) ((5))) ((4)) (3 4 5)))

In [47]:
(flatten s)

'(2 3 4 5 4 3 4 5)

In [48]:
(apply + (flatten s))

30

Now we define `add-tree` which adds up the elements of the tree using deep recursion which returns a single value.

In [49]:
(define (add-tree lst)
  (cond ((null? lst) 0)
    ((number? lst) lst)
    ((list? lst)
     (+ (add-tree (car lst))
        (add-tree (cdr lst))))
    ))

In [50]:
(add-tree s)

30

In [51]:
(trace add-tree)

In [52]:
(add-tree s)

>(add-tree '((2 3 (4) ((5))) ((4)) (3 4 5)))
> (add-tree '(2 3 (4) ((5))))
> >(add-tree 2)
< <2
> >(add-tree '(3 (4) ((5))))
> > (add-tree 3)
< < 3
> > (add-tree '((4) ((5))))
> > >(add-tree '(4))
> > > (add-tree 4)
< < < 4
> > > (add-tree '())
< < < 0
< < <4
> > >(add-tree '(((5))))
> > > (add-tree '((5)))
> > > >(add-tree '(5))
> > > > (add-tree 5)
< < < < 5
> > > > (add-tree '())
< < < < 0
< < < <5
> > > >(add-tree '())
< < < <0
< < < 5
> > > (add-tree '())
< < < 0
< < <5
< < 9
< <12
< 14
> (add-tree '(((4)) (3 4 5)))
> >(add-tree '((4)))
> > (add-tree '(4))
> > >(add-tree 4)
< < <4
> > >(add-tree '())
< < <0
< < 4
> > (add-tree '())
< < 0
< <4
> >(add-tree '((3 4 5)))
> > (add-tree '(3 4 5))
> > >(add-tree 3)
< < <3
> > >(add-tree '(4 5))
> > > (add-tree 4)
< < < 4
> > > (add-tree '(5))
> > > >(add-tree 5)
< < < <5
> > > >(add-tree '())
< < < <0
< < < 5
< < <9
< < 12
> > (add-tree '())
< < 0
< <12
< 16
<30


30

We next define a deep recursive function which transforms the tree, returning a modified version of the tree. We assume that the tree contains only numeric leaves.

In [53]:
(define (treeadd1 tree)
  (cond ((null? tree) '())
    ((not (pair? tree))
     (+ 1 tree))
    (else
     (cons (treeadd1 (car tree))
           (treeadd1 (cdr tree))))))

In [54]:
(treeadd1 s)

'((3 4 (5) ((6))) ((5)) (4 5 6))

In [55]:
(trace treeadd1)

In [56]:
(treeadd1 s)

>(treeadd1 '((2 3 (4) ((5))) ((4)) (3 4 5)))
> (treeadd1 '(2 3 (4) ((5))))
> >(treeadd1 2)
< <3
> >(treeadd1 '(3 (4) ((5))))
> > (treeadd1 3)
< < 4
> > (treeadd1 '((4) ((5))))
> > >(treeadd1 '(4))
> > > (treeadd1 4)
< < < 5
> > > (treeadd1 '())
< < < '()
< < <'(5)
> > >(treeadd1 '(((5))))
> > > (treeadd1 '((5)))
> > > >(treeadd1 '(5))
> > > > (treeadd1 5)
< < < < 6
> > > > (treeadd1 '())
< < < < '()
< < < <'(6)
> > > >(treeadd1 '())
< < < <'()
< < < '((6))
> > > (treeadd1 '())
< < < '()
< < <'(((6)))
< < '((5) ((6)))
< <'(4 (5) ((6)))
< '(3 4 (5) ((6)))
> (treeadd1 '(((4)) (3 4 5)))
> >(treeadd1 '((4)))
> > (treeadd1 '(4))
> > >(treeadd1 4)
< < <5
> > >(treeadd1 '())
< < <'()
< < '(5)
> > (treeadd1 '())
< < '()
< <'((5))
> >(treeadd1 '((3 4 5)))
> > (treeadd1 '(3 4 5))
> > >(treeadd1 3)
< < <4
> > >(treeadd1 '(4 5))
> > > (treeadd1 4)
< < < 5
> > > (treeadd1 '(5))
> > > >(treeadd1 5)
< < < <6
> > > >(treeadd1 '())
< < < <'()
< < < '(6)
< < <'(5 6)
< < '(4 5 6)
> > (treeadd1 '())
< < '(

'((3 4 (5) ((6))) ((5)) (4 5 6))

Finally, we define `double-tree` which copies a tree, replacing each leaf with twice its value.

In [57]:
;; assuming only numeric leaves to the tree
(define (double-tree tree)
  (cond
   [(null? tree) '()]
   [(not (pair? tree)) (+ tree tree)]
   [else
    (cons (double-tree (car tree))
          (double-tree (cdr tree)))]))

In [58]:
(double-tree s)

'((4 6 (8) ((10))) ((8)) (6 8 10))

In [59]:
(trace double-tree)

In [60]:
(double-tree s)

>(double-tree '((2 3 (4) ((5))) ((4)) (3 4 5)))
> (double-tree '(2 3 (4) ((5))))
> >(double-tree 2)
< <4
> >(double-tree '(3 (4) ((5))))
> > (double-tree 3)
< < 6
> > (double-tree '((4) ((5))))
> > >(double-tree '(4))
> > > (double-tree 4)
< < < 8
> > > (double-tree '())
< < < '()
< < <'(8)
> > >(double-tree '(((5))))
> > > (double-tree '((5)))
> > > >(double-tree '(5))
> > > > (double-tree 5)
< < < < 10
> > > > (double-tree '())
< < < < '()
< < < <'(10)
> > > >(double-tree '())
< < < <'()
< < < '((10))
> > > (double-tree '())
< < < '()
< < <'(((10)))
< < '((8) ((10)))
< <'(6 (8) ((10)))
< '(4 6 (8) ((10)))
> (double-tree '(((4)) (3 4 5)))
> >(double-tree '((4)))
> > (double-tree '(4))
> > >(double-tree 4)
< < <8
> > >(double-tree '())
< < <'()
< < '(8)
> > (double-tree '())
< < '()
< <'((8))
> >(double-tree '((3 4 5)))
> > (double-tree '(3 4 5))
> > >(double-tree 3)
< < <6
> > >(double-tree '(4 5))
> > > (double-tree 4)
< < < 8
> > > (double-tree '(5))
> > > >(double-tree 5)
< < < <10

'((4 6 (8) ((10))) ((8)) (6 8 10))

In [61]:
(pair? (cons 1 2))

#t

In [62]:
(list? (cons 1 2))

#f

In [63]:
(list? (cons 1 (cons 2 empty)))

#t

In [64]:
empty

'()